![](https://europe-west1-atp-views-tracker.cloudfunctions.net/working-analytics?notebook=tutorials--agent-security-with-qualifire--2-agent-security-with-qualifire-ui)

# Qualifireを使用したエージェントガードレール 🔥

このノートブックでは、AIエージェントにガードレールを追加するためのQualifireの統合方法を説明します。OpenAIのGPT-4.1とStreamlitを使用して簡単なチャットボットを構築し、GatewayとSDKの両方を使用してプロンプトインジェクション、安全でないコンテンツ、幻覚、ポリシー違反に対する保護を実装します。

## 概要

AIエージェントがより洗練され、広く展開されるようになるにつれて、その安全で制御された動作を確保することが重要になります。適切なガードレールがないと、AIエージェントはプロンプトインジェクション、安全でないコンテンツの生成、幻覚、ポリシー違反など、さまざまなリスクに対して脆弱になる可能性があります。

## ガードレール使用の利点

- **セキュリティの強化**: プロンプトインジェクション攻撃と不正アクセスから保護
- **コンテンツの安全性**: 有害または不適切なコンテンツの生成をフィルタリングして防止
- **品質管理**: 幻覚を減らし、事実に基づいた応答を確保
- **ポリシーコンプライアンス**: 組織のポリシーと使用ガイドラインを実施
- **リアルタイム保護**: 実行時に動作するガードレールを実装

## 主要なメソッド

1. **ゲートウェイ保護**: APIレベルのセキュリティ制御を実装
2. **コンテンツフィルタリング**: コンテンツモデレーションと安全性チェックを設定
3. **幻覚防止**: 事実確認と検証を追加
4. **ポリシー施行**: 使用ポリシーを設定して適用

ℹ️ デモアプリの作成方法に興味がある場合は、このチュートリアルに進む前に[Streamlitチュートリアル](../agent-with-streamlit-ui/building-chatbot-notebook.ipynb)をご覧ください。

<img src="./assets/freddie-shield.png" width="200px" alt="Qualifire Shield Logo">


## 1. セットアップと要件

In [ ]:
!pip install -q -r requirements.txt

### 1.2. Qualifireに登録してAPIキーを取得

ℹ️ すでにAPIキーを作成済みの場合は、このステップをスキップできます。

続行する前に、QualifireアカウントとAPIキーを持っていることを確認してください。

1. [https://app.qualifire.ai](https://app.qualifire.ai?utm=agents-towards-production)で登録します。
2. オンボーディングを完了し、APIキーを作成します。

<img src="./assets/protection-rules-evaluation.png" alt="Protection Rules Evaluation">
<img src="./assets/protection-rules-actions.png" alt="Protection Rules Actions">


3. 「ログを待機中」の画面が表示されたら、チュートリアルを続行できます。

<img src="./assets/wait-for-logs.png" alt="Waiting for logs">


## 2. Streamlitアプリのパッチ適用

## セットアップ: OpenAI APIキー

OpenAIのAPIを使用するには、ライブラリが認証できるようにAPIキーを提供する必要があります。これを行う方法はいくつかあります：

1. **オプション1（推奨）**: システムの環境変数としてAPIキーを設定します（例：`OPENAI_API_KEY`）。これにより、キーがコードの外に保持されます。
   - Linux/Mac: ターミナルで`export OPENAI_API_KEY='your_key_here'`
   - Windows: コマンドプロンプトで`set OPENAI_API_KEY="your_key_here"`

2. **オプション2**: コードでAPIキーを直接割り当てる（テスト用には便利ですが、コードを共有する場合はキーを公開しないよう注意してください）

このチュートリアルでは、安全のためにキーを環境変数として保存したと仮定します。秘密情報をハードコーディングしないことがベストプラクティスです。

ℹ️ 任意のLLMを使用できます。このチュートリアルでは、OpenAIのGPT-4.1を使用します。各LLMの具体的な設定を読みたい場合は、[ドキュメント](https://docs.qualifire.ai?utm=agents-towards-production)を確認してください。

### 2.1 Qualifireをゲートウェイとしてフック

In [ ]:
# app.py (すべてのステップを組み合わせた完全なコード)


########################################################
# オプションで、エージェントへの可観測性を得るためにトレーシングを追加できます
########################################################

import os
import qualifire
from dotenv import load_dotenv

load_dotenv()  # .envから環境変数を読み込む

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
QUALIFIRE_API_KEY = os.getenv("QUALIFIRE_API_KEY")
qualifire.init(
    api_key=QUALIFIRE_API_KEY,
)


########################################################
# 残りのコード
########################################################


import openai
import streamlit as st
import io

import openai
import os
import PyPDF2



if not OPENAI_API_KEY or not QUALIFIRE_API_KEY:
    raise ValueError("OPENAI_API_KEYとQUALIFIRE_API_KEYを設定する必要があります")


########################################################
# Streamlitアプリのパッチ適用
########################################################


client = openai.OpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://proxy.qualifire.ai/api/providers/openai",
    default_headers={
        "X-Qualifire-Api-Key": QUALIFIRE_API_KEY,
    },
)



########################################################
# app.pyコードの残り
########################################################

...

# 3. 新しい保護ルールの作成

Qualifireでは、保護ルールはLLM呼び出しに適用されるポリシーを定義する方法であり、ガードレールを適用し、期待どおりに動作しないLLMのエスケープハッチを設定できます。

1. 保護ルール[タブ](https://app.qualifire.ai/rules?utm=agents-towards-production)に移動します
2. ルールに名前を付けて「次へ」をクリックします
3. 評価を選択し、入力または出力で実行するかを選択します（今のところ入力のみ）
4. 評価が失敗したときに取るアクションを選択します。この例では、「それはできません」というデフォルトの応答を追加します
5. 「作成」をクリックします

#### 作業中のエージェントと対話できるようになったので、ガードレールをバイパスしてみましょう

1. 初期のジェイルブレイクとプロンプトインジェクションのセットには[これ](https://github.com/drorIvry/L1B3RT45/blob/main/OPENAI.mkd)を使用できます。
⚠️ これは参考資料です。自己責任で使用してください

Qualifireのジェイルブレイク保護ルールをオンとオフに切り替えて、LLMの応答にどのような影響があるかを確認してみてください。

In [ ]:
!streamlit run app.py

# <img src="./assets/prompt-injections-demo.gif">


# 3. まとめ

このチュートリアルを完了していただきありがとうございます！Qualifireを使用してエージェントの可観測性を向上させる方法を理解するのに役立ったことを願っています。

このチュートリアルでは、以下を学びました：

- 1行のコードでPythonアプリケーションでQualifireを初期化する方法
- バックグラウンドでOpenTelemetryを介して可観測性データを自動的にキャプチャするQualifireでエージェントを実行する方法
- LLM呼び出しを保護するためのQualifireガードレールの設定方法

Qualifireを使用すると、AIに対する深い比類のない制御、可視性、保護が提供されます。


### チュートリアルを完了していただきありがとうございます！ 🙏
Qualifireを始めるために、Proプランの1か月無料をご提供したいと思います。チェックアウト時にコード`NIR1MONTH`を使用してください

詳細については[https://qualifire.ai](https://qualifire.ai?utm=agents-towards-production)をご覧ください。